In [ ]:
!pip install transliterate
!pip install transformers datasets evaluate
!pip install googletrans==4.0.0-rc1
!pip install nltk

In [88]:
from transliterate import translit, get_available_language_codes
import string
from transformers import pipeline
from transformers import AutoTokenizer
import spacy
import random
from googletrans import Translator
import nltk
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
import difflib

In [89]:
generator = pipeline("text-generation", model="Zlovoblachko/test_L1_sent_generator")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [90]:
sents_dict = generator("It can be seen", num_return_sequences=4, return_full_text=True, do_sample=True, temperature=1)

In [91]:
sents_dict

[{'generated_text': 'It can be seen that in 2014 the biggest share of investment was in this area, namely in Italy. The percent of investment came in Russia, where in 2015 it was 5 billion Euros and in 2018 it is 8 billion Euros. Italy has the most'},
 {'generated_text': 'It can be seen in one chart that shows the total amount of visitors to different age groups over 10 years in N. Africa, Japan, Sweden, USA, Rest of World and the world using smartphone during period. The graph shows how many people use'},
 {'generated_text': 'It can be seen from the table what the rate of women and men in Japan (excepted in 2014-2015) in South Asia is the same: for the most part it is the same proportion (65%) and it is lower (14%)'},
 {'generated_text': 'It can be seen that the highest percentage of overweight children was in Italy in 2000. In Sweden is the youngest group, almost 15%. In contrast, obesity was almost the same, from 1980 to 2012. However, in USA the smallest percentage in overweight'}]

In [92]:
nlp = spacy.load("en_core_web_sm")
translator = Translator()
stemmer_en = PorterStemmer()
stemmer_ru = SnowballStemmer("russian")

In [93]:
def difference(string1, string2):
    affix = ""
    if len(string1) >= len(string2):
        return string1.replace(string2, '')
    else:
        return string2.replace(string1, '')

In [97]:
for i in sents_dict:
    doc = nlp(i['generated_text'])
    for j in doc.sents:
        flag = False
        j_text = j.text
        if j.text[-1] == '.':
            for token in j:
                if (token.pos_ == "NOUN") and flag == False and len(token.text) > 5:
                    if random.randint(0, 1) == 1:
                        if stemmer_en.stem(token.text) == token.text:
                            token_translit = translit(translator.translate(token.text, dest='ru').text, 'ru', reversed=True)
                            j_text = j_text.replace(token.text, "**"+token_translit+"**")
                            flag = True
                        else:
                            token_translit = translit(stemmer_ru.stem(translator.translate(token.text, dest='ru').text), 'ru', reversed=True)
                            affix = difference(token.text, stemmer_en.stem(token.text))
                            if len(affix) > 4:
                                j_text = j_text.replace(token.text, "**"+token_translit+"**")
                            else:
                                j_text = j_text.replace(token.text, "**"+token_translit+affix+"**")
                            flag = True
            if flag == True:
                print(j_text)

It can be seen that in 2014 the biggest share of **investitsment** was in this area, namely in Italy.
The percent of **investitsment** came in Russia, where in 2015 it was 5 billion Euros and in 2018 it is 8 billion Euros.
It can be seen in one chart that shows the total amount of **posetitels** to different age groups over 10 years in N. Africa, Japan, Sweden, USA, Rest of World and the world using smartphone during period.
It can be seen that the highest percentage of overweight **deti** was in Italy in 2000.
In contrast, **ozhirenity** was almost the same, from 1980 to 2012.
